In [2]:
%config Completer.use_jedi = False

In [3]:
from bravado.client import SwaggerClient

cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/api-docs',
                                config={"validate_requests":False,"validate_responses":False})
print(cbioportal)

SwaggerClient(https://www.cbioportal.org/api)


In [4]:
import pandas as pd

In [5]:
study_id='skcm_tcga_pan_can_atlas_2018'

In [6]:
def get_pandas_df(list):
    return pd.DataFrame.from_dict([
            dict(
                {k:getattr(m,k) for k in m})
                    for m in list
                    ])

In [7]:
mol_profiles=cbioportal.Molecular_Profiles.getAllMolecularProfilesInStudyUsingGET(studyId=study_id).result()
mpdf = get_pandas_df(mol_profiles)
mpdf.head()

,datatype,description,genericAssayType,molecularAlterationType,molecularProfileId,name,pivotThreshold,showProfileInAnalysisTab,sortOrder,study,studyId
0,LOG2-VALUE,Protein expression measured by reverse-phase p...,None,PROTEIN_LEVEL,skcm_tcga_pan_can_atlas_2018_rppa,Protein expression (RPPA),NaN,False,None,None,skcm_tcga_pan_can_atlas_2018
1,Z-SCORE,"Protein expression, measured by reverse-phase ...",None,PROTEIN_LEVEL,skcm_tcga_pan_can_atlas_2018_rppa_Zscores,Protein expression z-scores (RPPA),NaN,True,None,None,skcm_tcga_pan_can_atlas_2018
2,DISCRETE,Putative copy-number from GISTIC 2.0. Values: ...,None,COPY_NUMBER_ALTERATION,skcm_tcga_pan_can_atlas_2018_gistic,Putative copy-number alterations from GISTIC,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018
3,CONTINUOUS,"mRNA Expression, RSEM (Batch normalized from I...",None,MRNA_EXPRESSION,skcm_tcga_pan_can_atlas_2018_rna_seq_v2_mrna,"mRNA Expression, RSEM (Batch normalized from I...",NaN,False,None,None,skcm_tcga_pan_can_atlas_2018
4,Z-SCORE,mRNA expression z-scores (RNA Seq V2 RSEM) com...,None,MRNA_EXPRESSION,skcm_tcga_pan_can_atlas_2018_rna_seq_v2_mrna_m...,mRNA expression z-scores relative to diploid s...,NaN,True,None,None,skcm_tcga_pan_can_atlas_2018


In [8]:
mol_profile_id = mpdf.loc[mpdf['name'] == 'Mutations']['molecularProfileId'].item()
mol_profile_id

'skcm_tcga_pan_can_atlas_2018_mutations'

In [9]:
genes = cbioportal.Genes.getAllGenesUsingGET(keyword='BRAF').result()
gdf = get_pandas_df(genes)
gdf.head()

,entrezGeneId,geneticEntityId,hugoGeneSymbol,type
0,673,None,BRAF,protein-coding
1,286494,None,BRAFP1,pseudo
2,-11087,None,BRAF_PS147,phosphoprotein
3,-11088,None,BRAF_PS151,phosphoprotein
4,-59030,None,BRAF_PS314,phosphoprotein


In [10]:
entrez_gene_id = gdf.loc[gdf['hugoGeneSymbol'] == 'BRAF']['entrezGeneId'].item()
entrez_gene_id

673

In [11]:
mutations = cbioportal.Mutations.getMutationsInMolecularProfileBySampleListIdUsingGET(
    sampleListId='skcm_tcga_pan_can_atlas_2018_cnaseq', 
    molecularProfileId=mol_profile_id,
    entrezGeneId=entrez_gene_id).result()

mdf = get_pandas_df(mutations)
mdf.columns

Index(['alleleSpecificCopyNumber', 'aminoAcidChange', 'center', 'chr',
       'driverFilter', 'driverFilterAnnotation', 'driverTiersFilter',
       'driverTiersFilterAnnotation', 'endPosition', 'entrezGeneId',
       'fisValue', 'functionalImpactScore', 'gene', 'keyword', 'linkMsa',
       'linkPdb', 'linkXvar', 'molecularProfileId', 'mutationStatus',
       'mutationType', 'ncbiBuild', 'normalAltCount', 'normalRefCount',
       'patientId', 'proteinChange', 'proteinPosEnd', 'proteinPosStart',
       'referenceAllele', 'refseqMrnaId', 'sampleId', 'startPosition',
       'studyId', 'tumorAltCount', 'tumorRefCount', 'uniquePatientKey',
       'uniqueSampleKey', 'validationStatus', 'variantAllele', 'variantType'],
      dtype='object')

In [12]:
all_patients = mdf['patientId'].tolist()
len(set(all_patients))

191

In [13]:
altered_mdf = mdf[mdf['proteinChange']=='V600E']
altered_mdf.head()

,alleleSpecificCopyNumber,aminoAcidChange,center,chr,driverFilter,driverFilterAnnotation,driverTiersFilter,driverTiersFilterAnnotation,endPosition,entrezGeneId,...,sampleId,startPosition,studyId,tumorAltCount,tumorRefCount,uniquePatientKey,uniqueSampleKey,validationStatus,variantAllele,variantType
8,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-BF-AAP0-06,140453136,skcm_tcga_pan_can_atlas_2018,54,64,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1CRi1BQVAwLTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
9,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q4-06,140453136,skcm_tcga_pan_can_atlas_2018,26,29,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE0LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
10,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q5-06,140453136,skcm_tcga_pan_can_atlas_2018,33,17,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE1LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
11,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q6-06,140453136,skcm_tcga_pan_can_atlas_2018,17,36,VENHQS1EMy1BMVE2OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE2LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP
12,None,None,.,7,None,None,None,None,140453136,673,...,TCGA-D3-A1Q7-06,140453136,skcm_tcga_pan_can_atlas_2018,4,33,VENHQS1EMy1BMVE3OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,VENHQS1EMy1BMVE3LTA2OnNrY21fdGNnYV9wYW5fY2FuX2...,.,T,SNP


In [14]:
altered_patients = altered_mdf['patientId'].tolist()
len(set(altered_patients))

152

In [15]:
unaltered_patients = list(set(all_patients) - set(altered_patients))
len(unaltered_patients)

39

In [18]:
attribute_ids = ["OS_STATUS","OS_MONTHS","DFS_STATUS","DFS_MONTHS","DSS_STATUS","DSS_MONTHS","PFS_STATUS","PFS_MONTHS"]

clinical_data_single_study_filter = {
  'attributeIds': attribute_ids,
  'ids': altered_patients
}

altered_clinical = cbioportal.Clinical_Data.fetchAllClinicalDataInStudyUsingPOST(studyId = study_id, 
                                                              clinicalDataType='PATIENT',
                                                              clinicalDataSingleStudyFilter=clinical_data_single_study_filter).result()
altered_clinical_df = get_pandas_df(altered_clinical)
altered_clinical_df.head()

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
0,None,DSS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999
1,None,DSS_STATUS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:ALIVE OR DEAD TUMOR FREE
2,None,OS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999
3,None,OS_STATUS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
4,None,PFS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999


In [25]:
#altered_clinical_df
altered_os = altered_clinical_df.loc[(altered_clinical_df.clinicalAttributeId == "OS_MONTHS") | (altered_clinical_df.clinicalAttributeId == "OS_STATUS")]
altered_os

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
2,None,OS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999
3,None,OS_STATUS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
8,None,OS_MONTHS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.04260775
9,None,OS_STATUS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
13,None,OS_MONTHS,TCGA-D3-A1Q5,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.56862938
...,...,...,...,...,...,...,...,...
870,None,OS_STATUS,TCGA-YD-A9TB,None,skcm_tcga_pan_can_atlas_2018,VENHQS1ZRC1BOVRCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
874,None,OS_MONTHS,TCGA-Z2-A8RT,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BOFJUOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,27.583259362
875,None,OS_STATUS,TCGA-Z2-A8RT,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BOFJUOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
880,None,OS_MONTHS,TCGA-Z2-AA3V,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BQTNWOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,15.977907090999999


In [19]:
clinical_data_single_study_filter = {
  'attributeIds': attribute_ids,
  'ids': unaltered_patients
}

unaltered_clinical = cbioportal.Clinical_Data.fetchAllClinicalDataInStudyUsingPOST(studyId = study_id, 
                                                              clinicalDataType='PATIENT',
                                                              clinicalDataSingleStudyFilter=clinical_data_single_study_filter).result()
unaltered_clinical_df = get_pandas_df(unaltered_clinical)
unaltered_clinical_df.head()

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
0,None,DSS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.029950357
1,None,DSS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DEAD WITH TUMOR
2,None,OS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.029950357
3,None,OS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DECEASED
4,None,PFS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.005325969000001


In [27]:
#altered_clinical_df
altered_os = altered_clinical_df.loc[(altered_clinical_df.clinicalAttributeId == "OS_MONTHS") | (altered_clinical_df.clinicalAttributeId == "OS_STATUS")]
altered_os

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
2,None,OS_MONTHS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,14.925863825999999
3,None,OS_STATUS,TCGA-BF-AAP0,None,skcm_tcga_pan_can_atlas_2018,VENHQS1CRi1BQVAwOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
8,None,OS_MONTHS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.04260775
9,None,OS_STATUS,TCGA-D3-A1Q4,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE0OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
13,None,OS_MONTHS,TCGA-D3-A1Q5,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVE1OnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,112.56862938
...,...,...,...,...,...,...,...,...
870,None,OS_STATUS,TCGA-YD-A9TB,None,skcm_tcga_pan_can_atlas_2018,VENHQS1ZRC1BOVRCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
874,None,OS_MONTHS,TCGA-Z2-A8RT,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BOFJUOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,27.583259362
875,None,OS_STATUS,TCGA-Z2-A8RT,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BOFJUOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
880,None,OS_MONTHS,TCGA-Z2-AA3V,None,skcm_tcga_pan_can_atlas_2018,VENHQS1aMi1BQTNWOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,15.977907090999999


In [75]:
#unaltered_clinical_df
unaltered_os = unaltered_clinical_df.loc[(unaltered_clinical_df.clinicalAttributeId == "OS_MONTHS") | (unaltered_clinical_df.clinicalAttributeId == "OS_STATUS")]
unaltered_os

,clinicalAttribute,clinicalAttributeId,patientId,sampleId,studyId,uniquePatientKey,uniqueSampleKey,value
2,None,OS_MONTHS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,17.029950357
3,None,OS_STATUS,TCGA-3N-A9WB,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdCOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DECEASED
8,None,OS_MONTHS,TCGA-3N-A9WC,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdDOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,66.475983825
9,None,OS_STATUS,TCGA-3N-A9WC,None,skcm_tcga_pan_can_atlas_2018,VENHQS0zTi1BOVdDOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
14,None,OS_MONTHS,TCGA-D3-A1QA,None,skcm_tcga_pan_can_atlas_2018,VENHQS1EMy1BMVFBOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,90.903113391
...,...,...,...,...,...,...,...,...
210,None,OS_STATUS,TCGA-W3-AA1V,None,skcm_tcga_pan_can_atlas_2018,VENHQS1XMy1BQTFWOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,1:DECEASED
215,None,OS_MONTHS,TCGA-YD-A9TA,None,skcm_tcga_pan_can_atlas_2018,VENHQS1ZRC1BOVRBOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,49.183022652
216,None,OS_STATUS,TCGA-YD-A9TA,None,skcm_tcga_pan_can_atlas_2018,VENHQS1ZRC1BOVRBOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,0:LIVING
221,None,OS_MONTHS,TCGA-YG-AA3O,None,skcm_tcga_pan_can_atlas_2018,VENHQS1ZRy1BQTNPOnNrY21fdGNnYV9wYW5fY2FuX2F0bG...,None,37.939310254


In [125]:
months = unaltered_os.loc[unaltered_os.clinicalAttributeId.str.contains("MONTH")][['patientId', 'value']]
status = unaltered_os.loc[unaltered_os.clinicalAttributeId.str.contains("STATUS")][['patientId', 'value']]
# 0:LIVING, 1:DECEASED
status['value'] = status['value'].apply(lambda x: x.split(":")[0])

# merge months/status and sort by numeric month values
uos = months.merge(status, on='patientId', suffixes=['_month','_status']) \
            .astype({'value_status': 'float32', 'value_month': 'float32'}) \
            .sort_values('value_month', ascending=False)
            


In [126]:
from lifelines import KaplanMeierFitter

kmf = KaplanMeierFitter() 
kmf.fit(uos['value_month'], uos['value_status'], label='Kaplan Meier Estimate')

<lifelines.KaplanMeierFitter:"Kaplan Meier Estimate", fitted with 37 total observations, 17 right-censored observations>

In [128]:
kme = kmf.cumulative_density_.reset_index()
kme['Kaplan Meier Estimate']

0     0.000000
1     0.000000
2     0.027778
3     0.027778
4     0.056373
5     0.084967
6     0.113562
7     0.142157
8     0.170752
9     0.199346
10    0.199346
11    0.229000
12    0.229000
13    0.259840
14    0.290680
15    0.321520
16    0.321520
17    0.353829
18    0.386137
19    0.420241
20    0.454344
21    0.488448
22    0.488448
23    0.524987
24    0.561527
25    0.561527
26    0.561527
27    0.561527
28    0.561527
29    0.561527
30    0.561527
31    0.634606
32    0.707684
33    0.707684
34    0.707684
35    0.707684
36    0.707684
Name: Kaplan Meier Estimate, dtype: float64

In [130]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=uos.value_month,
    y=kme['Kaplan Meier Estimate'],
    mode='lines+markers',
    text=uos.patientId
))

fig.show()
